## Syntax

The syntax is very simple and generic:

```
    # this is a comment
    model   ::=  stmt+
    stmt    ::=  call | block
    call    ::=  name args "\n"
    args    ::=  "(" arglist? ")"
    arglist ::=  arg ("," arg)*
    arg     ::=  atom ("=" atom)?
    block   ::=  deco? name args? ":" "\n" INDENT stmt+ DEDENT
    deco    ::=  "@" name args? "\n"
    atom    ::=  name | number | code | string
    name    ::=  /[a-z][a-z0-9_]*/i
    number  ::=  /[+-]?[0-9]+/
    code    ::=  /{.*}/ | /[.*]/  # simplified here (nesting is correctly handled)
    string  ::=  /".*"/ | /'.*'/ | /""".*"""/a | /'''.*'''/a   # just Python strings
```

Each `block` must be one of:
 * `if` / `then` / `?else` (which means that `else` is optional)
 * `choose` / `+either` (which means that `either` may be repeated)
 * `parallel` / `+besides`
 * `race` / `+against`
 * `retry`
 * `negate`
 * `task`

The correct sequencing of blocks is not enforced in the syntax but when the parse tree is built. Moreover, each block may actually have parameters nd a decorator (`@...` like in Python). It is not checked at parse time wether arguments and decorators are correctly used, instead this is let to the compiler.

## Implementation

Import parser and build a parse tree from souce code. 

In [1]:
from zinc.io.aspic import parse
src = """
retry :
    if :
        pos()
        comp(x=10, y=42, duration=100)
        race :
            traj()
        against:
            negate :
                obs()
    then :
        pose()
        keep()
    else :
        negate :
            avoid()
"""
tree = parse(src)

The parse tree can be displayed in compact (`repr`) or full (`dump`) form.

In [2]:
tree

model :
 + body[0] = retry :
 |    + body[0] = if :
 |    |    + body[0] = call :
 |    |    |    + name = 'pos'
 |    |    + body[1] = call :
 |    |    |    + name = 'comp'
 |    |    |    + kargs['x'] = const :
 |    |    |    |    + value = 10
 |    |    |    |    + type = 'int'
 |    |    |    + kargs['y'] = const :
 |    |    |    |    + value = 42
 |    |    |    |    + type = 'int'
 |    |    |    + kargs['duration'] = const :
 |    |    |    |    + value = 100
 |    |    |    |    + type = 'int'
 |    |    + body[2] = race :
 |    |    |    + body[0] = call :
 |    |    |    |    + name = 'traj'
 |    |    |    + against[0] = against :
 |    |    |    |    + body[0] = negate :
 |    |    |    |    |    + body[0] = call :
 |    |    |    |    |    |    + name = 'obs'
 |    |    + else = else :
 |    |    |    + body[0] = negate :
 |    |    |    |    + body[0] = call :
 |    |    |    |    |    + name = 'avoid'
 |    |    + then = then :
 |    |    |    + body[0] = call :
 |   

In [3]:
tree.dump()

model :
 + body[0] = retry :
 |    + body[0] = if :
 |    |    + body[0] = call :
 |    |    |    + name = 'pos'
 |    |    |    + largs = []
 |    |    |    + kargs = {}
 |    |    + body[1] = call :
 |    |    |    + name = 'comp'
 |    |    |    + largs = []
 |    |    |    + kargs['x'] = const :
 |    |    |    |    + value = 10
 |    |    |    |    + type = 'int'
 |    |    |    + kargs['y'] = const :
 |    |    |    |    + value = 42
 |    |    |    |    + type = 'int'
 |    |    |    + kargs['duration'] = const :
 |    |    |    |    + value = 100
 |    |    |    |    + type = 'int'
 |    |    + body[2] = race :
 |    |    |    + body[0] = call :
 |    |    |    |    + name = 'traj'
 |    |    |    |    + largs = []
 |    |    |    |    + kargs = {}
 |    |    |    + args = None
 |    |    |    + deco = None
 |    |    |    + against[0] = against :
 |    |    |    |    + body[0] = negate :
 |    |    |    |    |    + body[0] = call :
 |    |    |    |    |    |    + name = 'obs'

The `dump` above allows to see all the attributes of each node, even the empty/not used ones. For instance, each block has attributes `deco` and `args`, even if they have not been used in the parsed source.

The parse tree is a nest of `node` instances, each one has the following attributes and methods:
 * `tag` the kind of the node (or the block name if this node is for a block)
 * `_fields` the names of its attributes
 * `__setitem__(self, key, val)` to add a new attribute and record it to `_fields`
 * `__getitem__(self, key)` to retrieve an attribute in a way more convenient than using `getattr`
 * `__contains__(self, key)` to check whether it has an attribute
 * `__iter__(self)` to iterate over the attributes in `_field` provided as name/value pairs

In [4]:
"spam" in tree, "body" in tree, tree["body"] is tree.body

(False, True, True)

## Compilation

A very basic compiler is provided as `zinc.io.meta.Compiler`, it implements a node tranformer pattern similar to Python's standard `ast.NodeTransformer`:
 * method `visit` allows to process a node
 * it calls a method `visit_tag` if it exists, where `tag` is the node's tag
 * otherwise it calls method `generic_visit` that basically turns the node into a `dict`, recursing over its attributes
 
To implement a compiler, one may subclass `Compiler` adding appropriate `visit_*` methods for each kind of node.

In [5]:
from zinc.io.meta import Compiler
comp = Compiler()

In [6]:
comp.visit(tree)

{'model': {'body': [{'retry': {'body': [{'if': {'body': [{'call': {'name': 'pos',
           'largs': [],
           'kargs': {}}},
         {'call': {'name': 'comp',
           'largs': [],
           'kargs': {'x': {'const': {'value': 10, 'type': 'int'}},
            'y': {'const': {'value': 42, 'type': 'int'}},
            'duration': {'const': {'value': 100, 'type': 'int'}}}}},
         {'race': {'body': [{'call': {'name': 'traj',
              'largs': [],
              'kargs': {}}}],
           'args': None,
           'deco': None,
           'against': [{'against': {'body': [{'negate': {'body': [{'call': {'name': 'obs',
                    'largs': [],
                    'kargs': {}}}],
                 'args': None,
                 'deco': None}}],
              'args': None,
              'deco': None,
              'against': []}}]}}],
        'args': None,
        'deco': None,
        'else': {'else': {'body': [{'negate': {'body': [{'call': {'name': 'avoid',
           